# Исследование надёжности заёмщиков

Задача - исследовать, влияет ли семейное положение и количество детей на погашение кредита в срок. Исследование производится на основе статистических данных. Результаты будут использованы в кредитном отделе банка

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd #подключаем библиотеку pandas
loan_borrowers = pd.read_csv('/datasets/data.csv') #читаем файл
loan_borrowers.head(15) #выводим первые 15 строк датафрейма

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Согласно первичному анализу, в данных обнаружены следующие проблемы:
1. Для многих заемщиков указан отрицательный общий трудовой стаж в днях. Необходимо разобраться, почему данная проблема возникла, а также испрвить эту ошибку представления. Необходимо оповестить сотрудников отдела разработки о данной проблеме. Также кажется странным, что общий трудовой стаж в днях не является целочисленным, он должен быть округлен и приведен к целочисленому формату для удобства обработки и наглядности представления. 
2. В данных уже на данном этапе видны пропуски. В строке 12 есть сразу 2 пропуска: в столбцах days_employed и  total_income. Очевидно, здесь присутствует завиисимость отсутсивия данных о трудовом стаже и заработной плате
3. В столбце education присутствуют неявные дубликаты - одни и те же значения в разном регистре. Эти данные необходимо привести к единому регистру
4. В столбце total_income пристуствют пропуски, о чем говорилось в п.2. Сумма ежемесячного дохода в данном столбце представлена с точность до 5 знаков после запятой (в данном случе - точки), что не совсем адеквтно реальному представлению. Об этом необходимо сообщить разработчикам - дествительно ли такой формат представления соответсвует потребностям бизнеса? При необходимости формат данных в данном столбце можно привести к более наглядному представлению: для текущих целей анализа данных максимально удобен целочисленный формат
5. В столбце purpose также пристутсвют неявные дубликаты. Одни и те же цели записаны по-разному (например, "покупка жилой недвижимости" и "покупка жилья"). Данные в этом столбце требуют проработки и приведения к единому виду: категоризировать. Также необходимо сообщить о данной проблеме в отдел разработки 

# Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print('Общая информация о таблице для выявления того, где в столбцах есть пропущенные значения')
loan_borrowers.info() #выводим общую информацию о таблице, чтобы видеть общее количетство пропусков во всех столбцах
isnull_loan_borrowers = loan_borrowers.isnull() #создаем датафрейм для удобства проверки значений NaN
null_days_and_income = isnull_loan_borrowers.loc[(isnull_loan_borrowers['days_employed'] == True) & (isnull_loan_borrowers['total_income'] == True)] #проверяем, в каких строках days_employed и total_income принимают значения NaN

print('Количество строк, в которых пропущены значения одновременно в столбцах days_employed и total_income: {}'.format(null_days_and_income['days_employed'].count())) #Выводим количество строк, в которых пропущены значения одновременно в столбцах days_employed и total_income
#Проверяем, есть ли просто нулевые значения (0) в столбцах days_employed и total_income:
print()
print('Количество строк days_employed, с нулевыми значениями до обработки Nan: {}'.format(loan_borrowers.loc[loan_borrowers['days_employed'] == 0, 'days_employed'].count()))
print('Количество строк total_income, с нулевыми значениями до обработки Nan: {}'.format(loan_borrowers.loc[loan_borrowers['total_income'] == 0, 'total_income'].count()))
#Нулевых значений не найдено, можно для удобства расчетов заменить NaN на нули 
loan_borrowers['days_employed'] = loan_borrowers['days_employed'].fillna(0)
loan_borrowers['total_income'] = loan_borrowers['total_income'].fillna(0)
#Выводим общую информацию по обновленной таблице:
print()
print('Общая информация о таблице после замены пропусков')
loan_borrowers.info()
print()
print("выведем уникальные значения в остальных столбцах для выявления еще неявных пропусков или странных значений")
#Проверим, присутствуют ли где-то в таблице неявные пропущенные значения. Для этого выведем список всех уникальных значений в столбцах
print('children: {}'.format(loan_borrowers['children'].unique()))
print('dob_years: {}'.format(loan_borrowers['dob_years'].unique()))
print('education: {}'.format(loan_borrowers['education'].unique()))
print('education_id: {}'.format(loan_borrowers['education_id'].unique()))
print('family_status: {}'.format(loan_borrowers['family_status'].unique()))
print('family_status_id: {}'.format(loan_borrowers['family_status_id'].unique()))
print('gender: {}'.format(loan_borrowers['gender'].unique()))
print('debt: {}'.format(loan_borrowers['debt'].unique()))
print('purpose: {}'.format(loan_borrowers['purpose'].unique()))
print()
#В столбце dob_years есть уникальное значение 0. необходимо проверить все строки с этим значением
print('Количество строк со значением dob_years = 0: {}'.format(loan_borrowers.loc[loan_borrowers['dob_years'] == 0,'dob_years'].count()))
#выведем для ознакомления сами строки, где  dob_years = 0
loan_borrowers.loc[loan_borrowers['dob_years'] == 0].head(10)


Общая информация о таблице для выявления того, где в столбцах есть пропущенные значения
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
Количество строк, в которых пропущены значения одновременно в столбцах days_employed и total_income: 2174

Количество строк days_employed, с нулевыми значениями до обработки Nan: 0
Количество строк total_income, с нулевыми значениями до обр

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,0.000000,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,0.000000,жилье
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


**Вывод**

Пропуски существуют в столбцах days_employed и total_income (и только в них), и количество этих пропусков одинаково и равно 2174 (разница между 21525 и 19351). Посчитав количество строк, в которых пропущены значения одновременно в столбцах days_employed и total_income выясняем, что их количество также равно 2174, то есть пропуски существуют только одновременно там, и там. Это явно неслучайное событие: очевидно, что такие пропуски появляются там, где клиент не может предоставить данные о своей финансовой надежности
Рассмотрим, какие данные хранятся в этих столбцах. 
days_employed - трудовой стаж в днях
total_income - месячная заработная плата.
Пропущенные значения в данных столбцах могут повлиять на качество полученного ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?", однако, во всех остальных столбцах данные содержатся, и их можно использовать для получения ответов на остальные вопросы (а количество таких строк составляет практически 10% от размера всей таблицы - не слишком разумно их удалять). Таким образом, данные строки необходимо оставть, заменив их значения на более удобные для обработки (При поиске ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" мы просто не будем их учитывать). Примем во внимание, что данные в этих столбцах - категориальные, хоть и записаны в числовом формате, то есть их удобно заменить некоторым значением по умолчанию. Для простоты и удобства можно заменить нулями. Убедившись, что во всех остальных строках в столбцах days_employed и total_income нет нулевых значений, заменяем все пропуски этими нулевыми значениями.
Для того, чтобы убедиться, нет ли неявных пропусков во всех остальных столбцах, выведем списки всех уникальных значений в этих столбцах. Кажется странным, что в 101 строке столбца dob_years указано значение 0: обычно младенцы не берут кредит. Оценив несклько таких строк, становится видно, что эти пропуски похожи на случайные. О них необходимо сообщить отделу разработки: скорее всего нули в данном столбце могли быть связаны с какой-то ошибкой. Строки содержат значения, которые могут быть необходимы для ответов на поставленные вопросы, при этом нет вопроса о зависимости возврата кредита в срок от возраста заемщика. Считаю необходимым эти строки оставить без изменений 

В предоставленных данных довольно много пропусков (более 10%), это довольно много, но на решение поставленных задач влиять не должно. Однако необходимо сообщить в кредитный отдел и отдел разработки о том, что такие пропуски есть - действительно ли могут выдать кредит человеку, по которому нет никаких данных о его финансовой надежности?

### Замена типа данных

In [3]:
def make_positive_numbers(number): #функция для преобразования отрицательных величин в положительные 
    result = number #присваиваем переменной, в которой бует храниться возвращаемый результат, значение параметра
    if number < 0: #если переданное значение отрицательно
        result = (-1) * number #делаем его положительным
    return result

print('Общая информация о датафрейме')
loan_borrowers.info()
loan_borrowers['days_employed'] = loan_borrowers['days_employed'].astype(int) #меняем тип данных с целочисленного на вещественный
loan_borrowers['total_income'] = loan_borrowers['total_income'].astype(int) #меняем тип данных с целочисленного на вещественный
print()
print('Общая о датафрейме после преобразования типов данных')
loan_borrowers.info()
#одновременно с заменой типов данных избавимся от отрицательных значений 
loan_borrowers['days_employed'] = loan_borrowers['days_employed'].apply(make_positive_numbers) #преобразуем все значения в столбце days_employed в положительные
loan_borrowers['children'] = loan_borrowers['children'].apply(make_positive_numbers) #преобразуем все значения в столбце children в положительные
loan_borrowers['total_income'] = loan_borrowers['total_income'].apply(make_positive_numbers) #преобразуем все значения в столбце total_income в положительные
loan_borrowers.head(10)

Общая информация о датафрейме
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

Общая о датафрейме после преобразования типов данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education   

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

В столбцах days_employed и total_income данные в исходном датафрейме представлены в формате float. Это не совсем удобно и не совсем логично - особенно для столбца days_employed. Для более правильного и удобного представления данные переведены в формат int методом astype, так как float и int - числовые форматы и перевод между ними осуществляется таким способом.
В столбцах days_employed, total_income и children избавились от отрицтельных значений (которые были обнаружены ранее).
Теперь все числовые данные представлены в формате, пригодном для правильной и наглядной обработки данных. 

### Обработка дубликатов

In [4]:
loan_borrowers['education'] = loan_borrowers['education'].str.lower() #Приводим данные в столбце educatuion к нижнему регистру
print('Количество дубликатов: {}'.format(loan_borrowers.duplicated().sum()))
loan_borrowers = loan_borrowers.drop_duplicates().reset_index(drop = True) #удаляем дубликаты
print('Количество дубликатов после удаления: {}'.format(loan_borrowers.duplicated().sum()))

Количество дубликатов: 71
Количество дубликатов после удаления: 0


**Вывод**

Ранее мы выводили список уникальных значений в каждом столбце, и было замечено, что в столбце education много повторяющихся значений в разных регистрах. Это может помешать поиску дубликатов. Все значения в данном столбце необходимо привести к единому регистру. Также об этом необходимо сообщить в отдел разработки, так как, судя по всему, эти данные берутся из справочника (есть education_id, и значений в education_id ровно столько, сколько действительно уникальных значений в education - 5): вполне возможно, есть какая-то ошибка представления данных: об этом необходимо сообщить в отдел разработки. Данные приводим к нижнему регистру методом .lower()
После приведения данных в столбце education к нижнему регистру ищем дубликаты методом duplicated(). Найден 71 дубликат. Дубликаты удалены, об их наличии также сообщено в отдел разработки.

### Лемматизация

In [5]:
from pymystem3 import Mystem #импортируем библиотеку с функциейй лемматизации
m = Mystem()
unique_purposes = loan_borrowers['purpose'].unique() #определяем уникальные значения в столбце purposes
purposes_lemmas = [] #создаем пустой список, куда будем записывать леммы леммы
for row in unique_purposes:
    lemma = m.lemmatize(row) #лемматизируем каждую строку unique_purposes
    purposes_lemmas.append(lemma) #добавляем полученную лемму в purposes_lemmas

purposes_lemmas #выводим на экран полученные леммы

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

**Вывод**

Для проведения лемматизации столбца с целями кредита, для начала получим все уникальные значения в этом столбце, после чего лемматизируем каждое полученное значение и запишем результат в purposes_lemmas
Согласно полученным данным лемматизации, можно понять, что цели получения кредита сводятся к следующим основным категориям:
-Свадьба
-Приобретение автомобиля
-Операции с недвижимостью (как с жилой, так и с коммерческой. ввиду того, что в некоторых целях не указано, какая именно недвижимость и какие цели преследует заемщик, целесообразно выделить просто одну категорию "недвижимость")
-Образование
Исходя из полученных сведений, будем производить категоризацию столбца purposes

### Категоризация данных

In [6]:
#Применим категоризацию данных к столбцу purpose: таким образом будет проще искать ответ на вопрос о влиянии целей кредита на возврат его в срок
def find_purp_category(purpose): #создаем функцию для определения категории каждой цели
    result = '' # определяем переменную, в которую запишем возвращаемый результат 
    try:
        lemma_purpose = m.lemmatize(purpose) #лемматизируем полученную строку
        for word in lemma_purpose: #проверяем каждое слово в лемматизированной строке
            word = word.lower() #на случай избежания оошибок, связанных с регистром, приводим слово к нижнему регистру
            #проверяем попадание слова в одну из определенных категорий
            if word == 'образование': 
                result = 'образование'
            elif word == 'свадьба':
                result = 'свадьба'
            elif word == 'автомобиль':
                result = 'автомобиль'
            elif word == 'недвижимость' or word == 'жилье':
                result = 'недвижимость'
    except:
        result = 'не определено' #если не получилось определить конкретную категорию из выбранного списка
    return result

#Для удобства дальнейшего анализа категоризируем столбец debt - факт наличия просроченной задолженности
def find_debt_category(debt): #функция для определения категории должника - есть долги по кредиту или нет
    result =''
    if debt == 0: #если значение параметра равно 0, то присваиваем значения "нет долгов"
        result = 'no debts'
    elif debt > 0:  #если значение параметра больше 0, то присваиваем значения "есть долги"
        result = 'debts'
    else:
        result = 'error - negative'  #обработка ошибки, если вдруг в столбце случайно появилось отрицательное значение
    return result       

#Для удобства анализа и представления в дальнейшем напишем функцию категоризации клиента в зависимости от количества детей у него
def find_children_category(children): #функция для определения категории клиента по признаку наличия детей
    result=''
    if children == 0: #если значение параметра равно 0, то присваиваем значения "нет детей"
        result = 'нет детей'
    elif children == 1 or children == 2:
        result = '1 или 2 ребенка'
    elif children > 2:
        result = 'многодетные'
    else:
        result = 'ошибка - отрицательное значение'
    return result
            


loan_borrowers['purpose_category'] = loan_borrowers['purpose'].apply(find_purp_category) #создаем столбец purpose_category, в который записываем результат применения функции find_purp_category (определяем категорию каждой цели)
loan_borrowers['debt_category'] = loan_borrowers['debt'].apply(find_debt_category) #создаем столбец debt_category, в который записываем результат применения функции find_debt_category (определяем категорию клиента - есть ли задолженность по кредиту)
loan_borrowers['children_category'] = loan_borrowers['children'].apply(find_children_category) #создаем столбец children_category, в который записываем результат применения функции find_children_category (определяем категорию клиента по наличию у него детей)
#Убеждаемся, что все категории определены:
print('Список уникальых значений столбца purpose_category: {}'.format(loan_borrowers['purpose_category'].unique())) #выводим список уникальных значений purpose_categoty, чтобы проверить, правильно ли отработала функция find_purp_category (нет ли неопределенных значений)
print('Список уникальых значений столбца debt_category: {}'.format(loan_borrowers['debt_category'].unique())) #выводим список уникальных значений purpose_categoty, чтобы проверить, правильно ли отработала функция find_purp_category (нет ли неопределенных значений)
print('Список уникальых значений столбца children_category: {}'.format(loan_borrowers['children_category'].unique())) #выводим список уникальных значений purpose_categoty, чтобы проверить, правильно ли отработала функция find_purp_category (нет ли неопределенных значений)
#Выводим контрольные 5 строк таблицы:
loan_borrowers.head(5)


Список уникальых значений столбца purpose_category: ['недвижимость' 'автомобиль' 'образование' 'свадьба']
Список уникальых значений столбца debt_category: ['no debts' 'debts']
Список уникальых значений столбца children_category: ['1 или 2 ребенка' 'нет детей' 'многодетные']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,debt_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,no debts,1 или 2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,no debts,1 или 2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,no debts,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,no debts,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,no debts,нет детей


**Вывод**

Проведена категориция целей кредита (столбец purpose_category): это позволит ответить на вопрос о зависимости возврата кредита в срок от цели кредита.
Также для удобства анализа проведена категоризация столбца debt - факт наличия просроченной задолженности: создан столбец debt_category - с ним представление аналитических данных при ответах на вопросы станет более нагладням
Проведена категоризация столбца children: children_category - также для целей анализа зависимомсти возврата кредита в срок от факта наличия детей (можно будет видеть по категории клиента: бездетный, 1-2 ребенка, многодетный)
Также для анализа зависимости возврата кредита в срок от уровня дохода необходима категоризация этого уровня дохода. Для удобства представления эта категоризация приведена в пункте ответа на вопрос "есть ли зависимость между уровнем дохода и возвратом кредита в срок?"

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
loan_borrowers_pivot_children = loan_borrowers.pivot_table(index = 'children', columns = 'debt_category', aggfunc = {'children':'count'}, fill_value = 0)  #создаем сводную таблицу, в которой будет видно количество заемщиков, у которых есть просроченная задолженность и у которых нет, сгрупптрованных по количеству детей
loan_borrowers_pivot_children['children','total_debts'] = loan_borrowers_pivot_children['children', 'debts'] + loan_borrowers_pivot_children['children', 'no debts'] #добавляем столбец, в котором будет подсчитано общее количество заемщиков, сгруппированных по количеству детей у них
loan_borrowers_pivot_children['children','percent_debts'] = loan_borrowers_pivot_children['children', 'debts'] / loan_borrowers_pivot_children['children', 'total_debts'] #добавляем столбец, в котором будет расчитана доля тех, у кого есть просроченная задолженность по каждой группе
print('количество и доля клиентов с просроченной здолженностью в зависимости от абсолютного количества детей')
print(loan_borrowers_pivot_children)
print()
#для наглядности делаем еще одну сводную таблицу, в которой данные будут выведены не по количеству детей, а по принадлежости клиента к определенной категории в зависимоти от того, сколько у него детей:
loan_borrowers_pivot_children_cat = loan_borrowers.pivot_table(index = 'children_category', columns = 'debt_category', aggfunc = {'children_category':'count'}, fill_value = 0)      
loan_borrowers_pivot_children_cat['children_category','total_debts'] = loan_borrowers_pivot_children_cat['children_category', 'debts'] + loan_borrowers_pivot_children_cat['children_category', 'no debts']
loan_borrowers_pivot_children_cat['children_category','percent_debts'] = loan_borrowers_pivot_children_cat['children_category', 'debts'] / loan_borrowers_pivot_children_cat['children_category', 'total_debts']
print('количество и доля клиентов с просроченной здолженностью в зависимости от количества детей (категория)')
print(loan_borrowers_pivot_children_cat)


количество и доля клиентов с просроченной здолженностью в зависимости от абсолютного количества детей
              children                                   
debt_category    debts no debts total_debts percent_debts
children                                                 
0                 1063    13028       14091      0.075438
1                  445     4410        4855      0.091658
2                  194     1858        2052      0.094542
3                   27      303         330      0.081818
4                    4       37          41      0.097561
5                    0        9           9      0.000000
20                   8       68          76      0.105263

количество и доля клиентов с просроченной здолженностью в зависимости от количества детей (категория)
                  children_category                                   
debt_category                 debts no debts total_debts percent_debts
children_category                                                     
1 

**Вывод**

Как можно видеть из обеих сводных таблиц, наименьший процент тех, у кого есть долги по кредиту - среди тех, у кого нет детей (7,5%). С появлением детей (1 или 2) ребенка доля должников растет (9,2% и 9,5% соответственно). Это и логично - в случае возникновения финансовых трудностей родитель всегда поставит интересы ребенка выше интересов кредитной организации. Можно также заметить, что для многодетных семей этот показатель ниже, чем для семей, где 1-2 ребенка, однако и количество многодетных семей, участвующих в анализе, значително ниже количества семей, где 1-2 ребенка (456 против 6907), то есть объем выборки достаточно мал, и может не до конца отражать реальное положение дел. Тем более, что в число многодетных попадают те, у кого 5 детей - их всего 9 человек и среди них нет тех, у кого есть просроченая задолженность. Вероятно, что их просто очень мало (для семей, где есть дети, доля тех, у кого есть просроченная задолженность, колеблется от 8,2% до 10,5%, а для 9 человек это меньше 1, то есть вполне возможно, что должник в эту выборку просто не попал)
Таким образом, отвечая на вопрос "есть ли зависимость между наличием детей и возвратом кредита в срок ?", можно ответить, что да, она есть, и те, у кого есть дети, чаще попадают в список заемщиков с просроченной задолженностью.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
loan_borrowers_pivot_marital_status = loan_borrowers.pivot_table(index = 'family_status', columns = 'debt_category', aggfunc = {'family_status':'count'}, fill_value = 0) #создаем сводную таблицу, в которой будет видно количество заемщиков, у которых есть просроченная задолженность и у которых нет, сгрупптрованных по их семейному положению
loan_borrowers_pivot_marital_status['family_status','total_debts'] = loan_borrowers_pivot_marital_status['family_status', 'debts'] + loan_borrowers_pivot_marital_status['family_status', 'no debts'] #считаем общее количество заемщиков в каждой категории
loan_borrowers_pivot_marital_status['family_status','percent_debts'] = loan_borrowers_pivot_marital_status['family_status', 'debts'] / loan_borrowers_pivot_marital_status['family_status', 'total_debts'] #доля заемщиков с просроченной здолженностью по каждой категории
loan_borrowers_pivot_marital_status.sort_values(by=('family_status','percent_debts'), ascending = False, inplace = True) #сортировка таблицы по доле заемщиков с просроченной задолженностью
print('Факт возврата кредита в срок в зависимости от семейного положения:')
print(loan_borrowers_pivot_marital_status)
print()
loan_borrowers_pivot_marital_status_children = loan_borrowers.pivot_table(index = ['family_status','children_category'], columns = 'debt_category', aggfunc = {'family_status':'count','children_category':'count'}, fill_value = 0) #создаем сводную таблицу, в которой будет видно количество заемщиков, у которых есть просроченная задолженность и у которых нет, сгрупптрованных по их семейному положению
#Выведем данные, сгруппированные по семейному положению и количеству детей:
print('Данные, сгруппированные по семейному положению и количеству детей')
loan_borrowers_pivot_marital_status_children['children_category','total_debts'] = loan_borrowers_pivot_marital_status_children['children_category','debts'] + loan_borrowers_pivot_marital_status_children['children_category','no debts']
loan_borrowers_pivot_marital_status_children['children_category','percent_debts'] = loan_borrowers_pivot_marital_status_children['children_category','debts'] / loan_borrowers_pivot_marital_status_children['children_category','total_debts']
loan_borrowers_pivot_marital_status_children

Факт возврата кредита в срок в зависимости от семейного положения:
                      family_status                                   
debt_category                 debts no debts total_debts percent_debts
family_status                                                         
Не женат / не замужем           274     2536        2810      0.097509
гражданский брак                388     3763        4151      0.093471
женат / замужем                 931    11408       12339      0.075452
в разводе                        85     1110        1195      0.071130
вдовец / вдова                   63      896         959      0.065693

Данные, сгруппированные по семейному положению и количеству детей


children_category           \
debt_category                                       debts no debts   
family_status         children_category                              
Не женат / не замужем 1 или 2 ребенка                  61      468   
                      многодетные                       3       16   
                      нет детей                       210     2052   
в разводе             1 или 2 ребенка                  28      369   
                      многодетные                       2       12   
                      нет детей                        55      729   
вдовец / вдова        1 или 2 ребенка                  10       91   
                      многодетные                       0       11   
                      нет детей                        53      794   
гражданский брак      1 или 2 ребенка                 148     1195   
                      многодетные                      11       67   
                      нет детей                       229     2501   
женат / замужем       1 или 2 ребенка                 392     4145   
                      многодетные                      23      311   
                      нет детей                       516     6952   

                                        family_status           \
debt_category                                   debts no debts   
family_status         children_category                          
Не женат / не замужем 1 или 2 ребенка              61      468   
                      многодетные                   3       16   
                      нет детей                   210     2052   
в разводе             1 или 2 ребенка              28      369   
                      многодетные                   2       12   
                      нет детей                    55      729   
вдовец / вдова        1 или 2 ребенка              10       91   
                      многодетные                   0       11   
                      нет детей                    53      794   
гражданский брак      1 или 2 ребенка             148     1195   
                      многодетные                  11       67   
                      нет детей                   229     2501   
женат / замужем       1 или 2 ребенка             392     4145   
                      многодетные                  23      311   
                      нет детей                   516     6952   

                                        children_category                
debt_category                                 total_debts percent_debts  
family_status         children_category                                  
Не женат / не замужем 1 или 2 ребенка                 529      0.115312  
                      многодетные                      19      0.157895  
                      нет детей                      2262      0.092838  
в разводе             1 или 2 ребенка                 397      0.070529  
                      многодетные                      14      0.142857  
                      нет детей                       784      0.070153  
вдовец / вдова        1 или 2 ребенка                 101      0.099010  
                      многодетные                      11      0.000000  
                      нет детей                       847      0.062574  
гражданский брак      1 или 2 ребенка                1343      0.110201  
                      многодетные                      78      0.141026  
                      нет детей                      2730      0.083883  
женат / замужем       1 или 2 ребенка                4537      0.086401  
                      многодетные                     334      0.068862  
                      нет детей                      7468      0.069095

**Вывод**

Из полученных данных можем видеть, что возврат кредита в срок достаточно сильно зависит от семейного положения. Те, кто нахоится или находился в официальном браке гораздо чаще возвращают кредит в срок. Максимальная доля вовремя возвращенных кредитов приходится на группу вдвец/вдова: либо получаемого стахового возмещения, либо из-за того, что в выборку попали многодетные семьи, в которых все кредиты были возвращены в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
def find_income_type_category(total_income): #функция для категоризации уровня дохода интуитивным правилом: распределение по уровню дохода согласно мнения многих живых людей
    low_lewel = 50000
    middle_level = 100000
    high_level = 300000
    super_high_level = 500000
    rich_level = 1000000
    super_rich_level = 1500000
    
    
    if total_income <=  low_lewel:
        result = '1 - ЗП до 50000'
    elif total_income <= middle_level:
        result = '2 - ЗП 50000 - 100000'
    elif total_income <= high_level:
        result = '3 - ЗП 100000 - 300000'
    elif total_income <= super_high_level:
        result = '4 - ЗП 300000 - 500000'
    elif total_income <= rich_level:
        result = '5 - ЗП 500000 - 1000000'
    elif total_income <= super_rich_level:
        result = '6 - ЗП 1000000 - 1500000'
    else:
        result = '7 - ЗП свыше 1500000'
    return result
    

def find_income_type_category2(total_income): #функция для категоризации уровня дохода по негласному правилу многих обепеенных людей
    low_lewel = 150000
    middle_level = 300000
    high_level = 1000000
    
    
    if total_income <=  low_lewel:
        result = '1 - ЗП до 150000'
    elif total_income <= middle_level:
        result = '2 - ЗП 150000 - 300000'
    elif total_income <= high_level:
        result = '3 - ЗП 300000 - 1000000'
    else:
        result = '4 - ЗП свыше 1000000'
    return result

def find_income_type_category_fss(total_income): #функция категоризации дохода согласно мнению ФСС
    low_lewel = 60000
    middle_level = 90000
    high_level = 150000
    
    
    if total_income <=  low_lewel:
        result = '1 - ЗП до 60000'
    elif total_income <= middle_level:
        result = '2 - ЗП 60000 - 90000'
    elif total_income <= high_level:
        result = '3 - ЗП 90000 - 150000'
    else:
        result = '4 - ЗП свыше 150000'
    return result
  

#Категоризируем уровень дохода всеми способами:
loan_borrowers['income_category'] = loan_borrowers['total_income'].apply(find_income_type_category)
loan_borrowers['income_category2'] = loan_borrowers['total_income'].apply(find_income_type_category2)
loan_borrowers['income_category_fss'] = loan_borrowers['total_income'].apply(find_income_type_category_fss)


#для начала необходимо убрать все нулевые total_income (об этом говорилось в выводе к п.1)

loan_borrowers_total_income = loan_borrowers.loc[loan_borrowers['total_income'] > 0]

#Содаем сводную таблицу, где данные сгруппированы по признку категоризированного уровня дохода
loan_borrowers_pivot__total_income = loan_borrowers_total_income.pivot_table(index = 'income_category', columns = 'debt_category', aggfunc = {'income_category':'count'}, fill_value = 0) #создание сводной таблицы
loan_borrowers_pivot__total_income['income_category','total_debts'] = loan_borrowers_pivot__total_income['income_category', 'debts'] + loan_borrowers_pivot__total_income['income_category', 'no debts'] #Добавляем столбец total_debts: общее количество заемщиков в каждой категории
loan_borrowers_pivot__total_income['income_category','percent_debts'] = loan_borrowers_pivot__total_income['income_category', 'debts'] / loan_borrowers_pivot__total_income['income_category', 'total_debts'] #Вычисляем долю должников в каждой категории
#loan_borrowers_total_income['total_income'].value_counts()
print(loan_borrowers_pivot__total_income) #выводим таблицу на экран

#Содаем сводную таблицу, где данные сгруппированы по признку категоризированного уровня дохода по второму правилу категоризации
loan_borrowers_pivot__total_income2 = loan_borrowers_total_income.pivot_table(index = 'income_category2', columns = 'debt_category', aggfunc = {'income_category2':'count'}, fill_value = 0)
loan_borrowers_pivot__total_income2['income_category2','total_debts'] = loan_borrowers_pivot__total_income2['income_category2', 'debts'] + loan_borrowers_pivot__total_income2['income_category2', 'no debts']
loan_borrowers_pivot__total_income2['income_category2','percent_debts'] = loan_borrowers_pivot__total_income2['income_category2', 'debts'] / loan_borrowers_pivot__total_income2['income_category2', 'total_debts']

print(loan_borrowers_pivot__total_income2)
#Содаем сводную таблицу, где данные сгруппированы по признку категоризированного уровня дохода по правилу ФСС
loan_borrowers_pivot__total_income_fss = loan_borrowers_total_income.pivot_table(index = 'income_category_fss', columns = 'debt_category', aggfunc = {'income_category_fss':'count'}, fill_value = 0)
loan_borrowers_pivot__total_income_fss['income_category_fss','total_debts'] = loan_borrowers_pivot__total_income_fss['income_category_fss', 'debts'] + loan_borrowers_pivot__total_income_fss['income_category_fss', 'no debts']
loan_borrowers_pivot__total_income_fss['income_category_fss','percent_debts'] = loan_borrowers_pivot__total_income_fss['income_category_fss', 'debts'] / loan_borrowers_pivot__total_income_fss['income_category_fss', 'total_debts']

print(loan_borrowers_pivot__total_income_fss)

#выводим данные по артефактному значению: клиент с доходом свыше 1500000, у которого есть факт просроченной здолженности
print()
print('LДанные по клиенту с доходом свыше 1500000, у которого есть факт просроченной здолженности')
print(loan_borrowers.loc[(loan_borrowers['total_income'] > 1500000) & (loan_borrowers['debt'] == 1),('children','family_status','income_type','total_income','purpose_category')])

                         income_category                                   
debt_category                      debts no debts total_debts percent_debts
income_category                                                            
1 - ЗП до 50000                       23      349         372      0.061828
2 - ЗП 50000 - 100000                331     3760        4091      0.080909
3 - ЗП 100000 - 300000              1111    12295       13406      0.082873
4 - ЗП 300000 - 500000                92     1168        1260      0.073016
5 - ЗП 500000 - 1000000               12      185         197      0.060914
6 - ЗП 1000000 - 1500000               1       17          18      0.055556
7 - ЗП свыше 1500000                   1        6           7      0.142857
                        income_category2                                   
debt_category                      debts no debts total_debts percent_debts
income_category2                                                           
1 - ЗП до 15

**Вывод**

Исходя из полученных данных можно сделать вывод, что наиболее часто возвращают в срок кредит те, чей доход не превышает 60000: то есть наименее обеспеченные клиенты: для них, возможно, более важна их кредитная история, а также залог по кредиту.
Также кредит лучше возврщают те, у кого высокий уровень дохода: свыше 300000, причем, чем уровень дохода выше, тем меньше доля тех, кто не вернул кредит в срок (однако здесь есть 1 артефактное значение: на очень маленькой выборке с доходом свыше 1500000 (7 человек) есть 1, кто не вернул кредит в срок: либо это случайное попадание, либо это может быть связано с тем, что люди с таким доходом могут чаще совершать рискованные операции. Если рассмотреть данные отдельно по данному клиенту, можно увидеть, что кредит был взят с целью строительства недвижимости: а это, вероятно, очень большая сумма. вполне возможно в какой-то момент дела в бизнесе пошли не так, как планировалось, и доходы упали)
У клиентов, чей доход находится в интервале 600000-300000 наиболее высокая доля невозвратов в срок.
Таким образом, можно сделать следующий вывод о зависимости возврата кредита в срок от уровня дохода: наиболее ответственно к возврату кредита в срок подходят люди с достаточно низким уровнем дохода (до 60000), для людей с высоким уровнем дохода (свыше 300000) доля возвращенных в срок кредитов выше в сравнении с тем, чей доход в пределах 60000 - 300000: их доход достатчно высок для исполнения своих обязательств, однако для людей с очень высоким уровнем дохода (свыше  1500000 вероятность невозврата кредита в срок может быть связана с высокими рисками предпринимательской деятельности - источника дохода)

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
loan_borrowers_pivot_purposes = loan_borrowers.pivot_table(index = 'purpose_category', columns = 'debt_category', aggfunc = {'purpose_category':'count'}, fill_value = 0)
loan_borrowers_pivot_purposes['purpose_category', 'total_debts'] = loan_borrowers_pivot_purposes['purpose_category', 'debts'] + loan_borrowers_pivot_purposes['purpose_category', 'no debts']
loan_borrowers_pivot_purposes['purpose_category', 'percent_debts'] = loan_borrowers_pivot_purposes['purpose_category', 'debts'] / loan_borrowers_pivot_purposes['purpose_category', 'total_debts']
loan_borrowers_pivot_purposes.sort_values(by=('purpose_category','percent_debts'), ascending = False, inplace = True)
loan_borrowers_pivot_purposes

purpose_category                                   
debt_category               debts no debts total_debts percent_debts
purpose_category                                                    
автомобиль                    403     3903        4306      0.093590
образование                   370     3643        4013      0.092200
свадьба                       186     2138        2324      0.080034
недвижимость                  782    10029       10811      0.072334

**Вывод**

Из полученных данных можно сделать вывод, что наиболее часто в срок возвращаются кредиты, взятые на различные операции с недвижимостью (покупка\строительство жилой\коомерческой недвижимости): в принципе, это кажется достаточно логичным, потому что на данные кредиты относительно низкие процентные ставки и большой залог (часто сама недвижимость), также этот вид кредита является самым популярным (чуть более 50% от общего числа кредитов)
Вторым наиболее "надежным" для кредитной организации является кредит на свадьбу. Его не возвращают в срок только 8%, при этом он самый непопулярный среди всех 4 видов.
Чаще всего в срок не вовращаются кредиты, взятые на приобретение автомобиля и на получение образования: на 2% чаще, чем для кредитов на недвижимость.

## Шаг 4. Общий вывод

Согласно полученным данным, на возврат кредита в срок влияют практически все факторы. Если отвечать на вопрос, поставленный во главе проекта - влияет ли количество детей и семейное положение заемщика на факт погашения кредита в срок, то можно ответить, что да, влияют. Наиболее благоприятные в этом отношении группы заемщиков - те, кто находился (именно вдовы\вдовцы и разведенные) в официальном браке и у кого нет детей. Также высоки показатели возврата кредита в срок среди тех, кто находится в официальном браке. Те, у кого нет пары или кто живет в гражданском браке, а также те, у кого 1-2 ребенка, кредит в срок возвращают реже.
Наиболее надежным видом кредита является кредит на приобретение или строительство недвижимости, наименее надежны: кредит на приобретение автомобиля и кредит на образование - их берут довольно часто, а возвращают реже.
Если говорить о зависимости факта возврата кредита в срок от величины дохода, то можно сказать следующее: однозначно нужно учитывать остальные факторы: в первую очередь, правильнее говорить о доходе на человека в семье (ведь 100000 для неамужней/неженатого и без детей не то же самое, что 100000 для семейной пары с 3 детьми). Исходя из тех данных, что есть, такой анализ произвести невозможно, потому что нет сведений о заработке второго взрослго члена семьи. Но если анализировать то, что имеется, то можно сказать, что те, чей доход досточно низок (ниже 60000) наиболее ответственны. А те, чей доход выше 300000 рублей, имеют меньше финансовых проблем и чаще возвращают кредит в срок. Однако стоит учитывать цель и род деятельности человека6 люди со сверхвысокими доходами часто склонны к наиболее рискованным видам деятельности, поэтому могут возникнуть проблемы со своевременным возвратом кредита